In [2]:
from app.services.audio.audio import AudioSlicer
from pathlib import Path
import io
from app.settings import settings

from app.database_redis.connection import get_redis_client
import pandas as pd
from faster_whisper import WhisperModel

check_and_process_connections_interval_sec: 1.0


/tmp/ipykernel_11016/2276236526.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

path = f"/audio/test.webm"

In [4]:
redis_client = await get_redis_client(settings.redis_host, settings.redis_port, settings.redis_password)

In [5]:
model_size = "large-v3"
model = WhisperModel(model_size, device="cuda", compute_type="float16")

In [6]:
audio_slicer = await AudioSlicer.from_ffmpeg_slice(path, 0, 600)
slice_duration = audio_slicer.audio.duration_seconds
audio_data = await audio_slicer.export_data()

2024-05-25 08:01:19,886 - INFO - app.services.audio.audio - None


In [7]:
segments, _ = model.transcribe(io.BytesIO(audio_data), beam_size=5, vad_filter=True, word_timestamps=True)
segments = [s for s in list(segments)]

In [8]:
import json

In [9]:
with open('2505_test_transcribe.json','w') as f:
    f.write(json.dumps(segments))